In [ ]:
!pip install --quiet torch torchvision torchaudio
!pip install --quiet transformers datasets peft evaluate opacus


In [ ]:
import transformers
import datasets
import peft
import accelerate
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from tqdm.auto import tqdm
import evaluate
from peft import PromptTuningConfig, PrefixTuningConfig, LoraConfig, get_peft_model, TaskType
from opacus import PrivacyEngine

print("All libraries imported successfully!")


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "prajjwal1/bert-tiny"


tokenizer = AutoTokenizer.from_pretrained(model_name)


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
print("TinyBERT model and tokenizer loaded successfully!")


In [ ]:
!pip install --upgrade datasets



In [ ]:
from datasets import load_dataset

datasets_dict = {}

print("Downloading SST2...")
datasets_dict["sst2"] = load_dataset("glue", "sst2")
print("SST2 downloaded!")

print("Downloading QNLI...")
datasets_dict["qnli"] = load_dataset("glue", "qnli")
print("QNLI downloaded!")

print("Downloading MNLI...")
datasets_dict["mnli"] = load_dataset("glue", "mnli")
print("MNLI downloaded!")

print("Downloading QQP...")
datasets_dict["qqp"] = load_dataset("glue", "qqp")
print("QQP downloaded!")


In [ ]:
for task in ["sst2", "qnli", "mnli", "qqp"]:
    print(f"\nSample from {task.upper()}:\n", datasets_dict[task]['train'][0])


In [ ]:
def tokenize_function_sst2(example):
    return tokenizer(example["sentence"], padding="max_length", truncation=True, max_length=128)

def tokenize_function_qnli(example):
    return tokenizer(example["question"], example["sentence"], padding="max_length", truncation=True, max_length=128)

def tokenize_function_mnli(example):
    return tokenizer(example["premise"], example["hypothesis"], padding="max_length", truncation=True, max_length=128)

def tokenize_function_qqp(example):
    return tokenizer(example["question1"], example["question2"], padding="max_length", truncation=True, max_length=128)

tokenize_functions = {
    "sst2": tokenize_function_sst2,
    "qnli": tokenize_function_qnli,
    "mnli": tokenize_function_mnli,
    "qqp": tokenize_function_qqp,
}


In [ ]:
tokenized_datasets = {}
for task in ["sst2", "qnli", "mnli", "qqp"]:
    print(f"Tokenizing {task}...")
    tokenized_datasets[task] = datasets_dict[task].map(tokenize_functions[task], batched=True)
print("All datasets tokenized!")


In [ ]:
dataset_info = {
    "sst2": {"num_labels": 2, "eval_split": "validation"},
    "qnli": {"num_labels": 2, "eval_split": "validation"},
    "qqp": {"num_labels": 2, "eval_split": "validation"},
    "mnli": {"num_labels": 3, "eval_split": "validation_matched"},
}
batch_size = 16
num_epochs = 3
learning_rate = 2e-5
max_grad_norm = 1.0
noise_multiplier = 0.376

#Model Functions

In [ ]:
def get_soft_prompt_model(num_labels):
    prompt_config = PromptTuningConfig(
        task_type=TaskType.SEQ_CLS,
        num_virtual_tokens=20
    )
    base_model = AutoModelForSequenceClassification.from_pretrained(
        "prajjwal1/bert-tiny", num_labels=num_labels
    )
    return get_peft_model(base_model, prompt_config)

def get_prefix_model(num_labels):
    prefix_config = PrefixTuningConfig(
        task_type=TaskType.SEQ_CLS,
        num_virtual_tokens=20
    )
    base_model = AutoModelForSequenceClassification.from_pretrained(
        "prajjwal1/bert-tiny", num_labels=num_labels
    )
    return get_peft_model(base_model, prefix_config)

def get_lora_model(num_labels):
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["query", "value"]
    )
    base_model = AutoModelForSequenceClassification.from_pretrained(
        "prajjwal1/bert-tiny", num_labels=num_labels
    )
    return get_peft_model(base_model, lora_config)

def get_softprompt_lora_model(num_labels):
    prompt_config = PromptTuningConfig(
        task_type=TaskType.SEQ_CLS,
        num_virtual_tokens=20
    )
    base_model = AutoModelForSequenceClassification.from_pretrained(
        "prajjwal1/bert-tiny", num_labels=num_labels
    )
    model = get_peft_model(base_model, prompt_config)
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["query", "value"]
    )
    return get_peft_model(model, lora_config)

def get_prefix_lora_model(num_labels):
    prefix_config = PrefixTuningConfig(
        task_type=TaskType.SEQ_CLS,
        num_virtual_tokens=20
    )
    base_model = AutoModelForSequenceClassification.from_pretrained(
        "prajjwal1/bert-tiny", num_labels=num_labels
    )
    model = get_peft_model(base_model, prefix_config)
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["query", "value"]
    )
    return get_peft_model(model, lora_config)

def get_full_finetune_model(num_labels):
    return AutoModelForSequenceClassification.from_pretrained(
        "prajjwal1/bert-tiny", num_labels=num_labels
    )

def get_last_layer_finetune_model(num_labels):
    model = AutoModelForSequenceClassification.from_pretrained(
        "prajjwal1/bert-tiny", num_labels=num_labels
    )
    for name, param in model.named_parameters():
        if not name.startswith("classifier"):
            param.requires_grad = False
    return model


#Training function without DP (Epsilon = infinity)

In [ ]:
def train_model(model, train_dataloader, num_epochs=3, lr=2e-5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )
    progress_bar = tqdm(range(num_training_steps))
    model.train()
    for epoch in range(num_epochs):
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            batch["labels"] = batch.pop("label")
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)



#Training Function With DP(Epsilon = (8)

In [ ]:
def train_model_dp(model, train_dataloader, num_epochs=num_epochs, lr=learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    model.train()
    privacy_engine = PrivacyEngine()
    model, optimizer, train_dataloader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_dataloader,
        noise_multiplier=noise_multiplier,
        max_grad_norm=max_grad_norm,
    )
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )
    progress_bar = tqdm(range(num_training_steps))
    model.train()
    for epoch in range(num_epochs):
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            batch["labels"] = batch.pop("label")
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
    epsilon = privacy_engine.get_epsilon(delta=1e-5)
    print(f"(ε, δ)-DP with ε = {epsilon:.2f}, δ = 1e-5")
    return epsilon

#Evaluate Function

In [ ]:
def evaluate_model(model, eval_dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    metric = evaluate.load("accuracy")
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        batch["labels"] = batch.pop("label")
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
    final_score = metric.compute()
    return final_score["accuracy"]

#Main Loop for all methods and datasets (without DP when epsilon =infinity)

In [ ]:
methods = {
    "soft_prompt": get_soft_prompt_model,
    "prefix": get_prefix_model,
    "lora": get_lora_model,
    "full_finetuning": get_full_finetune_model,
    "last_layer_finetuning": get_last_layer_finetune_model,
    "soft_prompt_lora": get_softprompt_lora_model,
    "prefix_lora": get_prefix_lora_model
}

results = {}

for method_name, model_func in methods.items():
    print(f"\n=== Running {method_name.replace('_', ' ').title()} (No DP) ===")
    results[method_name] = {}
    for task, info in dataset_info.items():
        print(f"\n--- Dataset: {task.upper()} ---")
        tokenized_datasets[task].set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
        train_dataloader = DataLoader(tokenized_datasets[task]["train"], batch_size=batch_size, shuffle=True)
        eval_dataloader = DataLoader(tokenized_datasets[task][info["eval_split"]], batch_size=batch_size)
        model = model_func(info["num_labels"])
        model.print_trainable_parameters()
        # Adjust learning rate and epochs for each method if needed
        if method_name in ["soft_prompt", "prefix", "lora", "soft_prompt_lora", "prefix_lora"]:
            train_model(model, train_dataloader, num_epochs=10, lr=0.3)
        elif method_name == "last_layer_finetuning":
            train_model(model, train_dataloader, num_epochs=10, lr=0.01)
        else:
            train_model(model, train_dataloader, num_epochs=3, lr=2e-5)
        accuracy = evaluate_model(model, eval_dataloader)
        print(f"{task.upper()} {method_name.replace('_', ' ').title()} Validation Accuracy: {accuracy:.4f}")
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        results[method_name][task] = {
            "trainable_params": trainable_params,
            "accuracy": accuracy
        }
print("\n=== Results (No DP, ε = ∞) ===")
for method, res in results.items():
    print(f"\n{method.replace('_', ' ').title()}:")
    for task, vals in res.items():
        print(f"  {task.upper()}: Params={vals['trainable_params']}, Accuracy={vals['accuracy']:.4f}")


#Main loop for all methods and datasets(with DP when Epsilon =8)

In [ ]:

def run_all_methods_dp(tokenized_datasets):
    methods = {
        "soft_prompt": get_soft_prompt_model,
        "prefix": get_prefix_model,
        "lora": get_lora_model,
        "full_finetuning": get_full_finetune_model,
        "last_layer_finetuning": get_last_layer_finetune_model,
        "soft_prompt_lora": get_softprompt_lora_model,
        "prefix_lora": get_prefix_lora_model
    }
    results = {}
    for method_name, model_func in methods.items():
        print(f"\n=== Running {method_name.replace('_', ' ').title()} with DP ===")
        results[method_name] = {}
        for task, info in dataset_info.items():
            print(f"\n--- Dataset: {task.upper()} ---")
            # Strictly set format to only tensors for required columns
            tokenized_datasets[task].set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
            train_dataloader = DataLoader(tokenized_datasets[task]["train"], batch_size=batch_size, shuffle=True)
            eval_dataloader = DataLoader(tokenized_datasets[task][info["eval_split"]], batch_size=batch_size)
            # Debug: check batch types
            batch = next(iter(train_dataloader))
            print("Batch types:", {k: type(v) for k, v in batch.items()})
            model = model_func(info["num_labels"])
            #model.print_trainable_parameters()
            epsilon = train_model_dp(model, train_dataloader)
            accuracy = evaluate_model(model, eval_dataloader)
            print(f"{task.upper()} {method_name.replace('_', ' ').title()} Validation Accuracy: {accuracy:.4f}")
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            print(f"Trainable params: {trainable_params}")

            results[method_name][task] = {
                "trainable_params": trainable_params,
                "epsilon": epsilon,
                "accuracy": accuracy
            }
    return results
results = run_all_methods_dp(tokenized_datasets)
